## Overview

Python provides a logging system as a part of its *standard library*, so you can quickly add logging to your application. 

## The Logging Module

> The [logging module](https://realpython.com/python-logging-source-code/) in Python is a ready-to-use and powerful 
> module that is designed to meet the needs of *beginners as well as enterprise teams*.
> 
> It is used by most of the third-party Python libraries, so you can integrate your log messages with the ones from
> those libraries to produce a homogeneous log for your application.

To import it:
```python
import logging
```

With the logging module imported, you can use something called a “logger” to log messages that you want to see.

By default, there are **5 standard levels indicating the severity of events**.

Each has a *corresponding method* that can be used to log events at that level of severity.

The defined levels, in order of increasing severity, are the following:
* `DEBUG`
* `INFO`
* `WARNING`
* `ERROR`
* `CRITICAL`

The logging module provides you with a **default logger** that allows you to *get started without needing to do much configuration*.

The corresponding *methods* for each level can be called as shown in the following example:

In [1]:
import logging

logging.debug("This is a debug message")
logging.info("This is an info message")
logging.warning("This is a warning message")
logging.error("This is an error message")
logging.critical("This is a critical message")

ERROR:root:This is an error message
CRITICAL:root:This is a critical message


The output shows the severity level before each message along with `root`, which is **the name the logging module gives to its default logger**.


Notice that the `debug()` and `info()` messages didn’t get logged. This is because, *by default, the logging module logs the messages with a severity level of `WARNING` or above*.

You can change that by configuring the logging module to log events of all levels if you want.

You can also define your own severity levels by changing configurations, **but it is generally not recommended** as it can cause confusion with logs of some third-party libraries that you might be using.

## Basic Configurations

You can use the `basicConfig(**kwargs)` method to configure the logging:

Fun fact:
> You will notice that the logging module breaks PEP8 styleguide and uses camelCase naming conventions.
> This is because it was adopted from `Log4j`, a logging utility in Java.
> It is a known issue in the package but by the time it was decided to add it to the standard library,
> it had already been adopted by users and changing it to meet PEP8 requirements would cause backwards compatibility issues.

Some of the commonly used parameters for `basicConfig()` are the following:
* `level`: The root logger will be set to the specified severity level.
* `filename`: This specifies the file.
* `filemode`: If filename is given, the file is opened in this mode. The default is `a`, which means append.
* `format`: This is the format of the log message.

In [1]:
# NOTE: Restart the notebook kernel before running this cell to see the effect!

import logging

logging.basicConfig(level=logging.DEBUG)
logging.debug("This will get logged")  # NOTE that debug level is now logged.

DEBUG:root:This will get logged


For logging to a file rather than the console, filename and filemode can be used, and you can decide the format of the message using format.

The following example shows the usage of all three:

In [1]:
# NOTE: Restart the notebook kernel before running this cell to see the effect!

import logging

logging.basicConfig(
    filename="app.log", filemode="w", format="%(name)s - %(levelname)s - %(message)s"
)
logging.warning("This will get logged to a file")

You can customize the root logger even further by using more parameters for `basicConfig()`,
which can be found [here](https://docs.python.org/3/library/logging.html#logging.basicConfig).

> ⚠️
>
> It should be noted that calling `basicConfig()` to configure the root logger works only if the root logger has not been configured before.
>
> **Basically, this function can only be called once.**

`debug()`, `info()`, `warning()`, `error()`, and `critical()` also call `basicConfig()` without arguments automatically if it has not been called before.

⚠️ This means that after the first time one of the above functions is called, you can no longer configure the root logger because they would have called the `basicConfig()` function internally.

### Formatting the Output

In [1]:
# NOTE: Restart the notebook kernel before running this cell to see the effect!

import logging

logging.basicConfig(format="%(process)d-%(levelname)s-%(message)s")  # NOTE.
logging.warning("This is a Warning")

641943-WARNING-This is a Warning


`format` can take a string with `LogRecord` attributes in any arrangement you like.

The entire list of available attributes can be found [here](https://docs.python.org/3/library/logging.html#logrecord-attributes).

In [1]:
# NOTE: Restart the notebook kernel before running this cell to see the effect!

import logging

logging.basicConfig(format="%(asctime)s - %(message)s", level=logging.INFO)
logging.info("Admin logged in")

2022-12-29 17:38:28,563 - Admin logged in


`%(asctime)s` adds the time of creation of the `LogRecord`.

The format can be changed using the `datefmt` attribute, which uses the same formatting language as the formatting functions in the `datetime` module, such as `time.strftime()`:

You can find the guide [here](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior).

In [2]:
# NOTE: Restart the notebook kernel before running this cell to see the effect!

import logging

logging.basicConfig(format="%(asctime)s - %(message)s", datefmt="%d-%b-%y %H:%M:%S")
logging.warning("Admin logged out")

2022-12-29 17:39:18,023 - Admin logged out


### Logging Variable Data

In most cases, you would want to include dynamic information from your application in the logs.

You have seen that the logging methods take a string as an argument, and it might seem natural to
format a string with variable data in a separate line and pass it to the log method.

**But this can actually be done directly by using a format string for the message and appending the variable data as arguments.**

Here’s an example:

In [3]:
# NOTE: Restart the notebook kernel before running this cell to see the effect!

import logging

name = "John"

logging.error(
    "%s raised an error", name
)  # NOTE the formatting is NOT the typical f-string approach.

2022-12-29 17:41:07,669 - John raised an error


In [1]:
# NOTE: Restart the notebook kernel before running this cell to see the effect!

# Can use f-strings if desired - but note potential performance impact.

import logging

name = "John"

logging.error(f"{name} raised an error")

ERROR:root:John raised an error


### Capturing Stack Traces

The logging module also allows you to capture the *full stack traces in an application*.

**Exception information can be captured if the `exc_info` parameter is passed as `True`, and the logging functions are called like this:**

In [2]:
import logging

a = 5
b = 0

try:
    c = a / b
except Exception as e:
    logging.error("Exception occurred", exc_info=True)  # NOTE, this is NECESSARY!

ERROR:root:Exception occurred
Traceback (most recent call last):
  File "/tmp/ipykernel_657246/1673112338.py", line 7, in <module>
    c = a / b
ZeroDivisionError: division by zero


If `exc_info` is not set to `True`, the output of the above program would not tell us anything about the exception,
which, in a real-world scenario, might not be as simple as a `ZeroDivisionError`.

Imagine trying to debug an error in a complicated codebase with a log that shows only this:
```
ERROR:root:Exception occurred
```

Here’s a quick tip: if you’re logging from an exception handler, **use the `logging.exception()` method**, which logs a
message with level `ERROR` and adds *exception information to the message*.

To put it more simply, calling `logging.exception()` is like calling `logging.error(exc_info=True)`.

But since this method always dumps exception information, it should only be called from an exception handler.

Take a look at this example:

In [3]:
import logging

a = 5
b = 0
try:
    c = a / b
except Exception as e:
    logging.exception(
        "Exception occurred"
    )  # NOTE. With .exception("...") we're always good!

ERROR:root:Exception occurred
Traceback (most recent call last):
  File "/tmp/ipykernel_657246/2953700927.py", line 6, in <module>
    c = a / b
ZeroDivisionError: division by zero


## Classes and Functions

So far, we have seen the default logger named `root`, which is used by the logging module whenever its functions are called directly like this: `logging.debug()`.

> You can (**and should**) define your *own logger* by creating an object of the `Logger` class,
> especially if your application has multiple modules\*.

\* i.e. if your application has more than one file, basically!

Let’s have a look at some of the classes and functions in the module.

The most commonly used classes defined in the logging module are the following:
* `Logger`: This is the class whose objects will be used in the application code directly to call the functions.
* `LogRecord`: Loggers automatically create `LogRecord` objects that have **all the information related to the event being logged**, like the name of the logger, the function, the line number, the message, and more.
* `Handler`: Handlers send the `LogRecord` *to the required output destination*, like the console or a file. Handler is a base for subclasses like `StreamHandler`, `FileHandler`, `SMTPHandler`, `HTTPHandler`, and more. These subclasses send the logging outputs to corresponding destinations, like `sys.stdout` or a disk file.
* `Formatter`: This is where you specify the format of the output by specifying a string format that lists out the attributes that the output should contain.

Out of these, we mostly deal with the objects of the `Logger` class,
> which are instantiated using the module-level function `logging.getLogger(name)`.

Multiple calls to `getLogger()` with the same name will return **a reference to the same `Logger` object**,
which saves us from passing the logger objects to every part where it’s needed.

Here’s an example:

In [4]:
import logging

logger = logging.getLogger("example_logger")
logger.warning("This is a warning")

> [❗] Unlike the `root` logger, a custom logger **can’t be configured using `basicConfig()`**.
>
> You have to configure it using `Handlers` and `Formatters`.

Also note this recommendation:
> It is recommended that we use module-level loggers by passing `__name__` as the name parameter to `getLogger()`
> to create a logger object as the *name of the logger itself would tell us from where the events are being logged*.
>
> `__name__` is a special built-in variable in Python which evaluates to the name of the current module.
>
> [Source](https://docs.python.org/3/library/logging.html#logger-objects)


## Using Handlers

Handlers come into the picture when you want to configure your own loggers and send the logs to multiple places when they are generated.

> A logger that you create **can have more than one handler**, which means you can set it up to be saved to a log file
> and also send it over email, ...

Like loggers, **you can also set the severity level in handlers**!
This is useful if you want to set multiple handlers for the same logger *but want different severity levels for each of them*.

In [5]:
# NOTE this whole recipe.
# Check file.log.

import logging

# Create a custom logger
logger = logging.getLogger(__name__)

# Create handlers
c_handler = logging.StreamHandler()
f_handler = logging.FileHandler("file.log")
c_handler.setLevel(logging.WARNING)
f_handler.setLevel(logging.ERROR)

# Create formatters and add it to handlers
c_format = logging.Formatter("%(name)s - %(levelname)s - %(message)s")
f_format = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
c_handler.setFormatter(c_format)
f_handler.setFormatter(f_format)

# Add handlers to the logger
logger.addHandler(c_handler)
logger.addHandler(f_handler)

logger.warning("This is a warning")
logger.error("This is an error")

__main__ - WARNING - This is a warning
__main__ - ERROR - This is an error
ERROR:__main__:This is an error


### Other Configuration Methods

You can configure logging
* as shown above using the module and class functions
* or by creating a config *file* or a *dictionary* and loading it using `fileConfig()` or `dictConfig()` respectively.

These are useful in case you want to change your logging configuration in a running application.

Here’s an example file configuration:

```toml
[loggers]
keys=root,sampleLogger

[handlers]
keys=consoleHandler

[formatters]
keys=sampleFormatter

[logger_root]
level=DEBUG
handlers=consoleHandler

[logger_sampleLogger]
level=DEBUG
handlers=consoleHandler
qualname=sampleLogger
propagate=0

[handler_consoleHandler]
class=StreamHandler
level=DEBUG
formatter=sampleFormatter
args=(sys.stdout,)

[formatter_sampleFormatter]
format=%(asctime)s - %(name)s - %(levelname)s - %(message)s
```

> In the above file, there are two loggers, one handler, and one formatter.
>
> [❗] **After their names are defined**, they are configured by adding the words `logger`, `handler`, and `formatter` before their names separated by an **underscore** (`_`).

To load this config file, you have to use `fileConfig()`:

In [6]:
import logging
import logging.config

logging.config.fileConfig(fname="file.conf", disable_existing_loggers=False)  # NOTE.

# Get the logger specified in the file
logger = logging.getLogger(__name__)

logger.debug("This is a debug message")

2022-12-29 18:01:09,012 - __main__ - DEBUG - This is a debug message


* The path of the config file is passed as a parameter to the `fileConfig()` method,
* and the `disable_existing_loggers` parameter is used to keep or disable the loggers that are present when the function is called.
    * ⚠️ It defaults to `True` if not mentioned.

Here’s the same configuration in a YAML format for the dictionary approach:

```yaml
version: 1
formatters:
  simple:
    format: '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
handlers:
  console:
    class: logging.StreamHandler
    level: DEBUG
    formatter: simple
    stream: ext://sys.stdout
loggers:
  sampleLogger:
    level: DEBUG
    handlers: [console]
    propagate: no
root:
  level: DEBUG
  handlers: [console]
```

**Note how formatters, handlers and the logger are interconnected in the above config.**

Here’s an example that shows how to load config from a `yaml` file:

In [1]:
# NOTE: Restart the notebook kernel before running this cell to see the effect!

import logging
import logging.config
import yaml  # NOTE.

with open("config.yaml", "r") as f:
    config = yaml.safe_load(f.read())  # NOTE. This will load it in as a dict.
    logging.config.dictConfig(config)  # NOTE.

logger = logging.getLogger(__name__)

logger.debug("This is a debug message")

2022-12-29 18:13:09,264 - __main__ - DEBUG - This is a debug message
